# Electrical Symbol Detection - Training on Google Colab
Train ResNet50+FPN model with CIoU Loss for multi-class object detection

## 1. Setup Environment

In [3]:
import sys
import torch

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU: Not available (will use CPU)")

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.9.0+cu128
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


## 2. Mount Google Drive (for saving checkpoints)

In [5]:
import sys
import os

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        print("✓ Google Drive mounted at /content/drive")
    except Exception as e:
        print(f"⚠ Could not mount Google Drive: {e}")
        print("Proceeding without Drive - checkpoints will save locally in /content/")
else:
    print("⚠ Running locally (not on Google Colab)")
    print("Dataset will be saved locally")

Mounted at /content/drive
✓ Google Drive mounted at /content/drive


## 3. Clone Repository

In [7]:
import os
import subprocess

repo_path = '/content/symbol-detection'

if not os.path.exists(repo_path):
    subprocess.run(['git', 'clone', 'https://github.com/BhanukaDev/symbol-detection.git', repo_path], check=True)
    print(f"Repository cloned to {repo_path}")
else:
    print(f"Repository already exists at {repo_path}")
    os.chdir(repo_path)
    subprocess.run(['git', 'pull'], check=True)
    print("Repository updated")

Repository already exists at /content/symbol-detection
Repository updated


## 4. Install Dependencies

In [ ]:
import os

# Change to python directory if in Colab
if os.path.exists('/content/symbol-detection/python'):
    os.chdir('/content/symbol-detection/python')
    
    # Install local workspace packages first
    print("Installing local workspace packages...")
    !pip install -e ./floor-grid
    !pip install -e ./effects
    print("✓ Workspace packages installed")

# Install external dependencies
!pip install torch torchvision torchmetrics pycocotools timm

# Install main package
!pip install -e .

Obtaining file:///content/symbol-detection/python
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached effects-0.0.1-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of symbol-detection to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement floor-grid (from symbol-detection) (from versions: none)
ERROR: No matching distribution found for floor-grid


## 5. Verify Installation

In [ ]:
from symbol_detection.training import Trainer, CIoULoss
from symbol_detection.dataset.generator import COCODatasetGenerator

print("✓ symbol-detection package imported successfully")
print(f"✓ Trainer available")
print(f"✓ CIoU Loss available")

## 6. Mount Dataset Location

In [ ]:
import os
import sys
from pathlib import Path

# Determine paths based on environment
IN_COLAB = 'google.colab' in sys.modules
DRIVE_MOUNTED = os.path.exists('/content/drive/MyDrive') if IN_COLAB else False

if IN_COLAB and DRIVE_MOUNTED:
    # Save to Google Drive
    dataset_dir = '/content/drive/MyDrive/symbol-detection/dataset'
    checkpoints_dir = '/content/drive/MyDrive/symbol-detection/checkpoints'
    print("✓ Using Google Drive for storage")
elif IN_COLAB:
    # Fallback to temporary Colab storage
    dataset_dir = '/content/symbol-detection/dataset'
    checkpoints_dir = '/content/symbol-detection/checkpoints'
    print("⚠ Google Drive not mounted - using temporary Colab storage")
else:
    # Local development
    dataset_dir = str(Path.cwd().parent.parent / 'python' / 'dataset')
    checkpoints_dir = str(Path.cwd().parent.parent / 'python' / 'checkpoints')
    print("📁 Using local storage")

os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)

print(f"Dataset directory: {dataset_dir}")
print(f"Checkpoints directory: {checkpoints_dir}")
print(f"Dataset exists: {os.path.exists(os.path.join(dataset_dir, 'annotations.json'))}")

## 7. Generate Dataset (Optional - if not using pre-generated)

In [ ]:
# Skip this if you already have dataset_dir/annotations.json
# Uncomment to generate dataset on Colab

# from symbol_detection.dataset.generator import COCODatasetGenerator
# 
# generator = COCODatasetGenerator(
#     dataset_dir=dataset_dir,
#     rows_min=20, rows_max=40,
#     cols_min=20, cols_max=40,
#     cell_size_min=20, cell_size_max=25,
# )
# 
# generator.generate_dataset(
#     num_images=500,
#     apply_symbol_effects=True,
#     apply_image_effects=True,
# )
# print(f"Dataset generated: {len(os.listdir(os.path.join(dataset_dir, 'images')))} images")

## 8. Training Configuration

In [ ]:
# Training hyperparameters
# Adjust batch_size based on GPU:
# - T4 (12GB): batch_size=4
# - A100 (40GB): batch_size=8-16
# - V100 (32GB): batch_size=8

training_config = {
    'num_epochs': 10,  # Reduced for testing
    'batch_size': 4,   # Reduced for T4 GPU
    'learning_rate': 0.005,
    'num_classes': 7,  # Electrical symbols
    'use_ciou_loss': True,  # Use Complete IoU Loss
}

print("Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")
print("\n⚠️ Using 10 epochs for quick test. Increase to 50+ for production training")

## 9. Run Training

In [ ]:
from symbol_detection.training import Trainer
import torch

# Initialize trainer
trainer = Trainer(
    dataset_dir=dataset_dir,
    output_dir=checkpoints_dir,
    num_classes=training_config['num_classes'],
    batch_size=training_config['batch_size'],
    learning_rate=training_config['learning_rate'],
    num_epochs=training_config['num_epochs'],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    use_ciou_loss=training_config['use_ciou_loss'],
)

print(f"Trainer initialized on device: {trainer.device}")
print(f"Model: FasterRCNN with ResNet50+FPN backbone")
print(f"CIoU Loss: {training_config['use_ciou_loss']}")

In [ ]:
# Start training
try:
    trainer.train()
    print("\n✓ Training completed successfully")
except Exception as e:
    print(f"✗ Training failed: {e}")
    import traceback
    traceback.print_exc()

## 10. Visualize Training Metrics

In [ ]:
import matplotlib.pyplot as plt
import json
from pathlib import Path

metrics_file = Path(checkpoints_dir) / 'metrics.json'

if metrics_file.exists():
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    plt.figure(figsize=(10, 6))
    plt.plot(metrics['train_losses'], label='Train Loss', marker='o')
    plt.plot(metrics['val_losses'], label='Validation Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{checkpoints_dir}/training_curve.png', dpi=150)
    plt.show()
    
    print(f"Final train loss: {metrics['train_losses'][-1]:.4f}")
    print(f"Final val loss: {metrics['val_losses'][-1]:.4f}")
else:
    print("Metrics file not found. Training may not have completed.")

## 11. List Saved Checkpoints

In [ ]:
from pathlib import Path

checkpoints = list(Path(checkpoints_dir).glob('*.pth'))

if checkpoints:
    print(f"Saved checkpoints ({len(checkpoints)}):")
    for ckpt in sorted(checkpoints):
        size_mb = ckpt.stat().st_size / (1024 * 1024)
        print(f"  {ckpt.name} ({size_mb:.1f} MB)")
    print(f"\nLatest checkpoint: {max(checkpoints, key=lambda x: x.stat().st_mtime).name}")
else:
    print("No checkpoints found")

## 12. Download Best Model (Optional)

In [ ]:
# The models are already saved in Google Drive (/content/drive/MyDrive/symbol-detection/checkpoints/)
# You can download them directly from Google Drive or use the Colab files interface

print(f"Checkpoints saved to: {checkpoints_dir}")
print("You can download them from Google Drive or use the Colab Files panel on the left")